<a href="https://colab.research.google.com/github/rr1989/capstone-project-recommendation-system-by-rr/blob/main/Final_calling_sentiment_model_plus_recommendation_system_by_Rakesh_Sept_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd


def sentiment_result(product_name,Pickled_RFC_Model):
  import pandas as pd
  #product_name = "Clorox Disinfecting Bathroom Cleaner"



  from sklearn.model_selection import cross_val_score
  from scipy.sparse import hstack
  from sklearn.feature_extraction.text import TfidfVectorizer
  Ebuss_df = pd.read_csv("sample30.csv")
  Ebuss_df_subset = Ebuss_df.dropna(subset=['reviews_text'])
  review_text=Ebuss_df_subset['reviews_text']
  #TF-IDF
  word_vectorizer = TfidfVectorizer(
  sublinear_tf=True,
  strip_accents='unicode',
  analyzer='word',
  token_pattern=r'\w{1,}',
  stop_words='english',
  ngram_range=(1, 1),
  max_features=10000)
  word_vectorizer.fit(review_text)
  char_vectorizer = TfidfVectorizer(
  sublinear_tf=True,
  strip_accents='unicode',
  analyzer='char',
  stop_words='english',
  ngram_range=(2, 6),
  max_features=50000)
  char_vectorizer.fit(review_text)

  index= Ebuss_df[Ebuss_df['name']==product_name].index.tolist()

  item_text = Ebuss_df.loc[index]['reviews_text']


  train_word_features = word_vectorizer.transform(item_text)

  train_char_features = char_vectorizer.transform(item_text)

  item_features = hstack([train_char_features, train_word_features])

  predicted_result = Pickled_RFC_Model.predict(item_features)

  if predicted_result.mean() > 4:
    return 1
  else:
    return 0


def recommendation_system(p_username):
    import pandas as pd
    # Load the Model back from file
    from sklearn.externals import joblib
    #Pkl_Filename = "Pickle_RFC1_Model.pkl"
    Pkl_Filename = "Pickle_XGB1_Model.pkl"

    Pickled_RFC_Model = joblib.load(Pkl_Filename)

    #Pickled_RFC_Model

    #recommendation system

    # import libraties
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt

    ratings = pd.read_csv('sample30.csv' , encoding='latin-1')
    #ratings.head()

    ratings = ratings[["reviews_username","name","reviews_rating"]]

    # Test and Train split of the dataset.
    from sklearn.model_selection import train_test_split
    train, test = train_test_split(ratings, test_size=0.30, random_state=31)

    train = train.groupby(by=["reviews_username",'name']).mean()

    train.reset_index(level=0, inplace=True)
    train.reset_index(level=0, inplace=True)

    # Item Based Similarity
    df_pivot = train.pivot(
    index='reviews_username',
    columns='name',
    values='reviews_rating').T

    #df_pivot.head()

    mean = np.nanmean(df_pivot, axis=1)
    df_subtracted = (df_pivot.T-mean).T

    from sklearn.metrics.pairwise import pairwise_distances

    # Item Similarity Matrix
    item_correlation = 1 - pairwise_distances(df_subtracted.fillna(0), metric='cosine')
    item_correlation[np.isnan(item_correlation)] = 0

    item_correlation[item_correlation<0]=0

    #prediction
    item_predicted_ratings = np.dot((df_pivot.fillna(0).T),item_correlation)

    # Copy the train dataset into dummy_train
    dummy_train = train.copy()

    # The products not rated by user is marked as 1 for prediction.
    dummy_train['reviews_rating'] = dummy_train['reviews_rating'].apply(lambda x: 0 if x>=1 else 1)

    # Convert the dummy train dataset into matrix format.
    dummy_train = dummy_train.pivot(
    index='reviews_username',
    columns='name',
    values='reviews_rating').fillna(1)

    item_final_rating = np.multiply(item_predicted_ratings,dummy_train)

    user_input = p_username

    d = item_final_rating.loc[user_input].sort_values(ascending=False)#[0:5]


    j = 0
    k = 0
    predicted_list = []
    original_list = []
    for i in d:
        predicted_result = sentiment_result(d.index[j],Pickled_RFC_Model)
        #print(d.index[j])
        #print(predicted_result)
        if predicted_result == 1:
            #print(d.index[j] + ' : ' + str(i))
            predicted_list.append(d.index[j])
            k = k + 1
            if k == 5:
              break
        original_list.append(d.index[j])
        j = j + 1
        #print(j)
    return d,predicted_list,original_list
        #return predicted_list

        


In [32]:
d,predicted_list,original_list = recommendation_system("06stidriver")

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:520: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'stop_words' will not be used"
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:520: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'stop_words' will not be used"
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:520: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'stop_words' will not be used"
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:520: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'stop_words' will not be used"
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:520: UserWarning: The para

In [33]:
d

name
Clorox Disinfecting Bathroom Cleaner                                              0.766556
Newman's Own Organics Licorice Twist, Black 5oz                                   0.022144
L'or233al Paris Elvive Extraordinary Clay Rebalancing Conditioner - 12.6 Fl Oz    0.019876
Sea Gull Lighting Six Light Bath Sconce/vanity - Brushed Nickel                   0.019500
Mrs. Meyer's174 Lemon Verbena Laundry Scent Booster - 18oz                        0.014756
                                                                                    ...   
Naturtint Nutrideep Multiplier Protective Cream                                   0.000000
Nature's Path Flax Plus Maple Pecan Crunch Cereal                                 0.000000
Nature's Path Chunky Chocolate Peanut Chewy Granola Bars                          0.000000
Musselman Apple Sauce, Cinnamon, 48oz                                             0.000000
0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest                     0.0

In [34]:
predicted_list

['Clorox Disinfecting Bathroom Cleaner',
 "Newman's Own Organics Licorice Twist, Black 5oz",
 "L'or233al Paris Elvive Extraordinary Clay Rebalancing Conditioner - 12.6 Fl Oz",
 'Sea Gull Lighting Six Light Bath Sconce/vanity - Brushed Nickel',
 "Mrs. Meyer's174 Lemon Verbena Laundry Scent Booster - 18oz",
 'Queen Helene Cocoa Butter Solid',
 'Dark Shadows (includes Digital Copy) (ultraviolet) (dvdvideo)',
 "Stacy's Simply Naked Bagel Chips",
 'Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)',
 'Pendaflex174 Divide It Up File Folder, Multi Section, Letter, Assorted, 12/pack',
 'Tresemme Kertatin Smooth Infusing Conditioning',
 'Iman Second To None Stick Foundation, Clay 1',
 'Vicks Vaporub, Regular, 3.53oz',
 'All,bran Complete Wheat Flakes, 18 Oz.',
 'Orajel Maximum Strength Toothache Pain Relief Liquid',
 'The Honest Company Laundry Detergent',
 "Meguiar's Deep Crystal Car Wash 64-Oz.",
 'The Script - No Sound Without Silence (cd)',
 'Tostitos Simply Blue Corn Tort

In [35]:
original_list

['Clorox Disinfecting Bathroom Cleaner',
 "Newman's Own Organics Licorice Twist, Black 5oz",
 "L'or233al Paris Elvive Extraordinary Clay Rebalancing Conditioner - 12.6 Fl Oz",
 'Sea Gull Lighting Six Light Bath Sconce/vanity - Brushed Nickel',
 "Mrs. Meyer's174 Lemon Verbena Laundry Scent Booster - 18oz",
 'Queen Helene Cocoa Butter Solid',
 'Dark Shadows (includes Digital Copy) (ultraviolet) (dvdvideo)',
 "Stacy's Simply Naked Bagel Chips",
 'Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)',
 'Pendaflex174 Divide It Up File Folder, Multi Section, Letter, Assorted, 12/pack',
 'Tresemme Kertatin Smooth Infusing Conditioning',
 'Iman Second To None Stick Foundation, Clay 1',
 'Nexxus Exxtra Gel Style Creation Sculptor',
 'Vicks Vaporub, Regular, 3.53oz',
 'All,bran Complete Wheat Flakes, 18 Oz.',
 'Orajel Maximum Strength Toothache Pain Relief Liquid',
 'The Honest Company Laundry Detergent',
 "Meguiar's Deep Crystal Car Wash 64-Oz.",
 'The Script - No Sound Without S

In [23]:
from sklearn.externals import joblib
Pkl_Filename = "Pickle_RFC_Model.pkl"

Pickled_RFC_Model = joblib.load(Pkl_Filename)

In [25]:
Pickled_RFC_Model

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)